In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

spark-2.4.0-bin-hadoop2.7/
spark-2.4.0-bin-hadoop2.7/python/
spark-2.4.0-bin-hadoop2.7/python/setup.cfg
spark-2.4.0-bin-hadoop2.7/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/resultiterable.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/heapq3.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/join.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/version.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/rdd.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/java_gateway.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/find_spark_home.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/_globals.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/worker.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/feature.py
spark-2.4.0-bin-hadoop2.7/python/pyspark

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
%cd '/content/gdrive/My Drive/LDS9_K273_ONLINE_BuiThiHuong/LDS9_K273_BuiThiHuong_Cuoi_ky/'

/content/gdrive/My Drive/LDS9_K273_ONLINE_BuiThiHuong/LDS9_K273_BuiThiHuong_Cuoi_ky


In [4]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf

import time

from pyspark.sql.types import DoubleType
from pyspark.sql.functions import isnan, when, count, col, udf
from pyspark.sql.functions import *

from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [5]:
# SparkContext.setSystemProperty('spark.executor.memory', '12g')
sc = SparkContext(master='local', appName='Recommendation_Amazon_Office_Products')

In [6]:
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession(sc)

## Read data

In [8]:
start_time = time.time()

In [9]:
data = spark.read.json('Data/Office_Products_5.json')

In [10]:
data.show(5,truncate=True)

+----------+-------+-------+--------------------+-----------+--------------+-------------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|       reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+-------------------+--------------------+--------------+
|B00000JBLH| [3, 4]|    5.0|I bought my first...| 09 3, 2004|A32T2H8150OJLU|                ARH|A solid performer...|    1094169600|
|B00000JBLH| [7, 9]|    5.0|WHY THIS BELATED ...|12 15, 2007|A3MAFS04ZABRGO|   Let it Be "Alan"|Price of GOLD is ...|    1197676800|
|B00000JBLH| [3, 3]|    2.0|I have an HP 48GX...| 01 1, 2011|A1F1A0QQP2XVH5|             Mark B|Good functionalit...|    1293840000|
|B00000JBLH| [7, 8]|    5.0|I've started doin...|04 19, 2006| A49R5DBXXQDE5|       R. D Johnson|One of the last o...|    1145404800|
|B00000JBLH| [0, 0]|    5.0|For simple calcul...| 08 4, 2013|A2XRMQA6

In [11]:
data.count()

53258

In [12]:
# choose suitable columns
data_sub = data.select(['asin', 'overall', 'reviewerID'])

In [13]:
data_sub.show(5)

+----------+-------+--------------+
|      asin|overall|    reviewerID|
+----------+-------+--------------+
|B00000JBLH|    5.0|A32T2H8150OJLU|
|B00000JBLH|    5.0|A3MAFS04ZABRGO|
|B00000JBLH|    2.0|A1F1A0QQP2XVH5|
|B00000JBLH|    5.0| A49R5DBXXQDE5|
|B00000JBLH|    5.0|A2XRMQA6PJ5ZJ8|
+----------+-------+--------------+
only showing top 5 rows



In [14]:
data_sub.count()

53258

In [15]:
data_sub.printSchema()

root
 |-- asin: string (nullable = true)
 |-- overall: double (nullable = true)
 |-- reviewerID: string (nullable = true)



In [16]:
# change data type
data_sub = data_sub.withColumn("overall", data_sub["overall"].cast(DoubleType()))

In [17]:
data_sub.select([count(
                        when(col(c).isNull(), c))
                        .alias(c) 
                        for c in data_sub.columns]).toPandas().T

,0
asin,0
overall,0
reviewerID,0


In [18]:
# Distinct users and movies
users = data_sub.select("reviewerID").distinct().count()
products = data_sub.select("asin").distinct().count()
numerator = data_sub.count()

In [19]:
display(numerator, users, products)

53258

4905

2420

In [20]:
# Number of ratings matrix could contain if no empty cells
denominator = users * products
denominator

11870100

In [21]:
#Calculating sparsity
sparsity = 1 - (numerator*1.0 / denominator)
print ("Sparsity: "), sparsity

Sparsity: 


(None, 0.9955132644206873)

## Pipeline

In [22]:
# Converting String to index
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

In [23]:
# # Create an indexer
# indexer = StringIndexer(inputCol='asin', 
#                         outputCol='asin_idx')

# # Indexer identifies categories in the data
# indexer_model = indexer.fit(data_sub)

# # Indexer creates a new column with numeric index values
# data_indexed = indexer_model.transform(data_sub)

# # Repeat the process for the other categorical feature
# indexer1 = StringIndexer(inputCol='reviewerID', 
#                          outputCol='reviewerID_idx')
# indexer1_model = indexer1.fit(data_indexed)
# data_indexed = indexer1_model.transform(data_indexed)

In [24]:
# data_indexed.show(5, truncate=True)

In [25]:
# data_indexed.select([count(when(col(c).isNull(), c)).alias(c) for c in 
#            data_indexed.columns]).toPandas().T

In [26]:
# # Smaller dataset so we will use 0.8 / 0.2
# (training, test) = data_indexed.randomSplit([0.8, 0.2])

In [27]:
# StringIndexer
indexer1 = StringIndexer(inputCol='asin',
                         outputCol='asin_idx', handleInvalid='skip')
indexer2 = StringIndexer(inputCol='reviewerID',
                         outputCol='reviewerID_idx', handleInvalid='skip')

# ALS model
als = ALS(maxIter=5,
          regParam=0.09,
          rank=25,
          userCol='reviewerID_idx',
          itemCol='asin_idx',
          ratingCol='overall',
          coldStartStrategy='drop',
          nonnegative=True)

In [28]:
# create pipeline
pipeline_als = Pipeline(stages=[indexer1, indexer2, als])

## Train, test spliting

In [29]:
# smaller dataset so we will use 0.8/0.2
(training, test) = data_sub.randomSplit([0.8,0.2])

In [30]:
training.count()

42599

In [31]:
test.count()

10659

## Build model

In [32]:
# als = ALS(maxIter=5,
#           regParam=0.01,
#           userCol='userId',
#           itemCol='movieId',
#           ratingCol='rating',
#           coldStartStrategy='drop',
#           nonnegative=True)

In [33]:
# fit model with training data
pipeline_als_model = pipeline_als.fit(training)

## Evaluation

In [34]:
# evaluate the model by computing the RMSE on the test data
predictions = pipeline_als_model.transform(test)

In [35]:
predictions.show(5)

+----------+-------+--------------+--------+--------------+----------+
|      asin|overall|    reviewerID|asin_idx|reviewerID_idx|prediction|
+----------+-------+--------------+--------+--------------+----------+
|B00D7NYJ2W|    5.0|A3ATKIU6MZZ4MU|   148.0|         988.0|  4.694685|
|B00D7NYJ2W|    5.0|A3RZOPQ42Q50B1|   148.0|        2322.0| 4.4306064|
|B00D7NYJ2W|    4.0| AK4HKI46S49NA|   148.0|        2013.0| 4.1653214|
|B00D7NYJ2W|    5.0|A2E7RX6AFUDQEX|   148.0|        1807.0| 4.1071496|
|B00D7NYJ2W|    5.0|A3J029KBNG5MNP|   148.0|        1250.0| 5.1327605|
+----------+-------+--------------+--------+--------------+----------+
only showing top 5 rows



In [36]:
evaluator = RegressionEvaluator(metricName='rmse',
                                labelCol='overall',
                                predictionCol='prediction')

In [37]:
rmse = evaluator.evaluate(predictions)

In [38]:
rmse

1.0385961222836608

In [39]:
data.describe().show()

+-------+----------+-----------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
|summary|      asin|          overall|          reviewText|reviewTime|          reviewerID|        reviewerName|             summary|      unixReviewTime|
+-------+----------+-----------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
|  count|     53258|            53258|               53258|     53258|               53258|               52799|               53258|               53258|
|   mean|      null|4.345957414848473|                null|      null|                null|               319.0|                null|1.3400844165383604E9|
| stddev|      null|0.930249873823577|                null|      null|                null|                 0.0|                null| 5.266063365635067E7|
|    min|B00000JBLH|              1.0|                    |01 1, 2005|

* RMSE = 1.064 > stddev = 0.93 => This model is not goood => Try to turning parameters

In [40]:
data.agg({'overall': 'mean'}).show()

+-----------------+
|     avg(overall)|
+-----------------+
|4.345957414848473|
+-----------------+



In [41]:
data.agg({'overall': 'std'}).show()

+-----------------+
|  stddev(overall)|
+-----------------+
|0.930249873823577|
+-----------------+



## Turning parameters

In [42]:
# ALS model with different parameters
als_t = ALS(maxIter=20,
          regParam=0.1,
          userCol='reviewerID_idx',
          itemCol='asin_idx',
          ratingCol='overall',
          coldStartStrategy='drop',
          nonnegative=True)
# if the capacity of computer is power enough => User RandomSearchCV or GridSearchCV
# if not, do manually one by one

In [43]:
# create pipeline
pipeline_als_t = Pipeline(stages=[indexer1, indexer2, als_t])

In [44]:
# fit model with training data
pipeline_als_model_t = pipeline_als_t.fit(training)

In [45]:
# evaluate the model by computing the RMSE on the test data
predictions_t = pipeline_als_model_t.transform(test)

In [46]:
predictions_t.show(5)

+----------+-------+--------------+--------+--------------+----------+
|      asin|overall|    reviewerID|asin_idx|reviewerID_idx|prediction|
+----------+-------+--------------+--------+--------------+----------+
|B00D7NYJ2W|    5.0|A3ATKIU6MZZ4MU|   148.0|         988.0|  4.866054|
|B00D7NYJ2W|    5.0|A3RZOPQ42Q50B1|   148.0|        2322.0|  4.201542|
|B00D7NYJ2W|    4.0| AK4HKI46S49NA|   148.0|        2013.0| 4.1488132|
|B00D7NYJ2W|    5.0|A2E7RX6AFUDQEX|   148.0|        1807.0|  3.990164|
|B00D7NYJ2W|    5.0|A3J029KBNG5MNP|   148.0|        1250.0| 5.2385197|
+----------+-------+--------------+--------+--------------+----------+
only showing top 5 rows



In [47]:
rmse_t = evaluator.evaluate(predictions_t)

In [48]:
rmse_t

1.0102708743735835

In [49]:
data.describe().show()

+-------+----------+-----------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
|summary|      asin|          overall|          reviewText|reviewTime|          reviewerID|        reviewerName|             summary|      unixReviewTime|
+-------+----------+-----------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
|  count|     53258|            53258|               53258|     53258|               53258|               52799|               53258|               53258|
|   mean|      null|4.345957414848473|                null|      null|                null|               319.0|                null|1.3400844165383604E9|
| stddev|      null|0.930249873823577|                null|      null|                null|                 0.0|                null| 5.266063365635067E7|
|    min|B00000JBLH|              1.0|                    |01 1, 2005|

* RMSE = 1.01 while stddev = 0.93 => This model is better.
* pipeline_als_model_t is better than pipeline_als because rmse_t is lower

## Save the results

### create user_recs dataframe

In [50]:
# extract the ALS model
model = pipeline_als_model_t.stages[-1]

In [51]:
# get 10 recommendations which have highest rating
user_recs = model.recommendForAllUsers(users)

In [52]:
user_recs.printSchema()

root
 |-- reviewerID_idx: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- asin_idx: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [53]:
for user in user_recs.head(3):
  print(user)

Row(reviewerID_idx=1580, recommendations=[Row(asin_idx=1781, rating=3.9936091899871826), Row(asin_idx=1860, rating=3.9579358100891113), Row(asin_idx=1968, rating=3.8949248790740967), Row(asin_idx=1152, rating=3.889150381088257), Row(asin_idx=2063, rating=3.8488285541534424), Row(asin_idx=1407, rating=3.8268485069274902), Row(asin_idx=1804, rating=3.820173740386963), Row(asin_idx=565, rating=3.7987213134765625), Row(asin_idx=1315, rating=3.796278715133667), Row(asin_idx=1357, rating=3.795503854751587), Row(asin_idx=381, rating=3.790114164352417), Row(asin_idx=2374, rating=3.7884504795074463), Row(asin_idx=2282, rating=3.7739009857177734), Row(asin_idx=1952, rating=3.7725892066955566), Row(asin_idx=357, rating=3.7703640460968018), Row(asin_idx=1103, rating=3.768329620361328), Row(asin_idx=2001, rating=3.765620231628418), Row(asin_idx=65, rating=3.759751081466675), Row(asin_idx=1390, rating=3.7590978145599365), Row(asin_idx=532, rating=3.745016574859619), Row(asin_idx=1528, rating=3.73927

In [54]:
user_recs.show(3, truncate=False)

+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [55]:
user_recs.count()

4905

### create asin_data dataframe

In [56]:
pipeline_als_model_t.stages[0:3]

[StringIndexer_c62ba3f9122e, StringIndexer_f59b57a6fb9f, ALS_32e62e4be5b6]

In [57]:
# check to ensure the asin_data contains all reviewerIDs
# transform "training" data by fitted indexer model
asin_data_train = pipeline_als_model_t.stages[0].transform(training).select('asin','asin_idx').distinct()
asin_data_train = asin_data_train.withColumnRenamed('asin_idx', 'asin_idx_train')
asin_data_train.count()
# transform "data" by fitted indexer model
asin_data_full = pipeline_als_model_t.stages[0].transform(data).select('asin','asin_idx').distinct()
asin_data_full = asin_data_full.withColumnRenamed('asin_idx', 'asin_idx_full')
asin_data_full.count()

combined_asin_data = asin_data_full.join(asin_data_train, on='asin')
combined_asin_data.show(5)
# compare
combined_asin_data = combined_asin_data.withColumn('check', col('asin_idx_full')==col('asin_idx_train'))
combined_asin_data.filter(col('check') != 'true').show()

+----------+-------------+--------------+
|      asin|asin_idx_full|asin_idx_train|
+----------+-------------+--------------+
|B00006IFH9|       1762.0|        1762.0|
|B00079VIFU|       2194.0|        2194.0|
|B000AN0UH0|        869.0|         869.0|
|B000SBXIA0|       1978.0|        1978.0|
|B0011N0LP4|       2387.0|        2387.0|
+----------+-------------+--------------+
only showing top 5 rows

+----+-------------+--------------+-----+
|asin|asin_idx_full|asin_idx_train|check|
+----+-------------+--------------+-----+
+----+-------------+--------------+-----+



* asin_data_train and asin_data_full are the same

In [58]:
# final asin_data
asin_data = pipeline_als_model_t.stages[0].transform(data).select('asin','asin_idx').distinct()
asin_data.show(5)

+----------+--------+
|      asin|asin_idx|
+----------+--------+
|B0000C3HI4|  2052.0|
|B0009WH87M|   938.0|
|B000S14PFM|  1331.0|
|B00144862U|   895.0|
|B004OA758C|  1025.0|
+----------+--------+
only showing top 5 rows



### create reviewer_data

In [59]:
pipeline_als_model_t.stages[0:3]

[StringIndexer_c62ba3f9122e, StringIndexer_f59b57a6fb9f, ALS_32e62e4be5b6]

In [60]:
# check to ensure the reviewerID_data contains all reviewerIDs
# transform "training" data by fitted indexer model
reviewerID_data_train = pipeline_als_model_t.stages[1].transform(training).select('reviewerID','reviewerID_idx').distinct()
reviewerID_data_train = reviewerID_data_train.withColumnRenamed('reviewerID_idx', 'reviewerID_idx_train')
reviewerID_data_train.count()
# transform "data" by fitted indexer model
reviewerID_data_full = pipeline_als_model_t.stages[1].transform(data).select('reviewerID','reviewerID_idx').distinct()
reviewerID_data_full = reviewerID_data_full.withColumnRenamed('reviewerID_idx', 'reviewerID_idx_full')
reviewerID_data_full.count()

combined_reviewerID_data = reviewerID_data_full.join(reviewerID_data_train, on='reviewerID')
combined_reviewerID_data.show(5)
# coompare
combined_reviewerID_data = combined_reviewerID_data.withColumn('check', col('reviewerID_idx_full')==col('reviewerID_idx_train'))
combined_reviewerID_data.filter(col('check') != 'true').show()

+--------------+-------------------+--------------------+
|    reviewerID|reviewerID_idx_full|reviewerID_idx_train|
+--------------+-------------------+--------------------+
|A17A1KTVI3DG6U|             1514.0|              1514.0|
|A1RBBQPFEY2WCI|             4468.0|              4468.0|
|A1VLE2SH9J8WYS|             4721.0|              4721.0|
|A2BMZRO0H7TFCS|               83.0|                83.0|
|A2P6QCZWW3H1X6|             2890.0|              2890.0|
+--------------+-------------------+--------------------+
only showing top 5 rows

+----------+-------------------+--------------------+-----+
|reviewerID|reviewerID_idx_full|reviewerID_idx_train|check|
+----------+-------------------+--------------------+-----+
+----------+-------------------+--------------------+-----+



* revewerID_data_train and revewerID_data_full are the same

In [61]:
# final reviewerID_data
reviewerID_data = pipeline_als_model_t.stages[1].transform(data).select('reviewerID','reviewerID_idx').distinct()
reviewerID_data.show(5)

+--------------+--------------+
|    reviewerID|reviewerID_idx|
+--------------+--------------+
| AZD8SMNGQI98O|         649.0|
|A2ZFFSJ0VM6KSJ|        3924.0|
|A2W4014S06NAIT|         840.0|
|A3TRHAU4UTBWJ6|        4611.0|
|A1FPQ54AIB26L0|         329.0|
+--------------+--------------+
only showing top 5 rows



## Recomendation for specific users

In [62]:
# test recommendation by a given user
reviewerId = 'A3CJ7MHAS9IMAM'

In [63]:
reviewerId_idx = reviewerID_data.filter(col('reviewerID')==reviewerId).select('reviewerID_idx').collect()[0][0]
reviewerId_idx

489.0

In [64]:
result = user_recs.filter(user_recs['reviewerID_idx']==reviewerId_idx)
result.show(truncate=False)

+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [65]:
result = result.select(result.reviewerID_idx, explode(result.recommendations))
result.show(5)

+--------------+-----------------+
|reviewerID_idx|              col|
+--------------+-----------------+
|           489|[1269, 5.3275876]|
|           489| [1231, 5.323815]|
|           489| [1351, 5.301122]|
|           489|[1579, 5.1558466]|
|           489|   [872, 5.15557]|
+--------------+-----------------+
only showing top 5 rows



In [66]:
# recommendation result
result = result.withColumn('asin_idx', result.col.getField('asin_idx'))\
                           .withColumn('rating', result.col.getField('rating'))
result = result.join(reviewerID_data, on = 'reviewerID_idx').join(asin_data, on='asin_idx')
result.show()

+--------+--------------+-----------------+---------+--------------+----------+
|asin_idx|reviewerID_idx|              col|   rating|    reviewerID|      asin|
+--------+--------------+-----------------+---------+--------------+----------+
|     299|           489| [299, 3.5371823]|3.5371823|A3CJ7MHAS9IMAM|B00A8YIAPA|
|     305|           489| [305, 3.8981123]|3.8981123|A3CJ7MHAS9IMAM|B009QYHAIQ|
|     496|           489| [496, 3.6377144]|3.6377144|A3CJ7MHAS9IMAM|B001R4C5BK|
|     558|           489| [558, 3.7472637]|3.7472637|A3CJ7MHAS9IMAM|B008R52BRA|
|     596|           489|  [596, 4.846043]| 4.846043|A3CJ7MHAS9IMAM|B002K9GPSA|
|     692|           489| [692, 4.5288153]|4.5288153|A3CJ7MHAS9IMAM|B00GDDV5UW|
|     769|           489|  [769, 4.386402]| 4.386402|A3CJ7MHAS9IMAM|B002X94T8U|
|     934|           489|  [934, 3.270253]| 3.270253|A3CJ7MHAS9IMAM|B0016ZQ566|
|    1051|           489| [1051, 3.982506]| 3.982506|A3CJ7MHAS9IMAM|B000K67ZE8|
|    1761|           489|[1761, 4.313885

In [67]:
# create a function to do recommendation
def recommendation_specific_user(userId):
  reviewerId = userId
  reviewerId_idx = reviewerID_data.filter(col('reviewerID')==reviewerId).select('reviewerID_idx').collect()[0][0]
  result = user_recs.filter(user_recs['reviewerID_idx']==reviewerId_idx)
  result = result.select(result.reviewerID_idx, explode(result.recommendations))
  result = result.withColumn('asin_idx', result.col.getField('asin_idx'))\
                           .withColumn('rating', result.col.getField('rating'))
  result = result.join(reviewerID_data, on = 'reviewerID_idx').join(asin_data, on='asin_idx')
  return result

In [68]:
# make recommendations to some users: A3CJ7MHAS9IMAM, A3LGT6UZL99IW1, A21KNRUAA5RK5E
lst_user = ['A3CJ7MHAS9IMAM','A3LGT6UZL99IW1', 'A21KNRUAA5RK5E']
for i in lst_user:
  print('Recommendations for user', i, ':')
  recommendation_specific_user(i).show()

Recommendations for user A3CJ7MHAS9IMAM :
+--------+--------------+-----------------+---------+--------------+----------+
|asin_idx|reviewerID_idx|              col|   rating|    reviewerID|      asin|
+--------+--------------+-----------------+---------+--------------+----------+
|     299|           489| [299, 3.5371823]|3.5371823|A3CJ7MHAS9IMAM|B00A8YIAPA|
|     305|           489| [305, 3.8981123]|3.8981123|A3CJ7MHAS9IMAM|B009QYHAIQ|
|     496|           489| [496, 3.6377144]|3.6377144|A3CJ7MHAS9IMAM|B001R4C5BK|
|     558|           489| [558, 3.7472637]|3.7472637|A3CJ7MHAS9IMAM|B008R52BRA|
|     596|           489|  [596, 4.846043]| 4.846043|A3CJ7MHAS9IMAM|B002K9GPSA|
|     692|           489| [692, 4.5288153]|4.5288153|A3CJ7MHAS9IMAM|B00GDDV5UW|
|     769|           489|  [769, 4.386402]| 4.386402|A3CJ7MHAS9IMAM|B002X94T8U|
|     934|           489|  [934, 3.270253]| 3.270253|A3CJ7MHAS9IMAM|B0016ZQ566|
|    1051|           489| [1051, 3.982506]| 3.982506|A3CJ7MHAS9IMAM|B000K67ZE8

## Recomendation by threshold

In [69]:
# # filter all movieIds having rationg >= 3.0
# result.filter(result.rating >= 3.0).show()

## Save results

In [70]:
# save to parquets
user_recs.write.parquet('C5_Recommendation_Amazon_Office_Products_user_recs.parquet', mode='overwrite')
reviewerID_data.write.parquet('C5_Recommendation_Amazon_Office_Products_reviewerID.parquet', mode='overwrite')
asin_data.write.parquet('C5_Recommendation_Amazon_Office_Products_asin.parquet', mode='overwrite')

In [71]:
# read parquets
user_recs = spark.read.parquet('C5_Recommendation_Amazon_Office_Products_user_recs.parquet')
reviewerID_data = spark.read.parquet('C5_Recommendation_Amazon_Office_Products_reviewerID.parquet')
asin_data = spark.read.parquet('C5_Recommendation_Amazon_Office_Products_asin.parquet')

In [72]:
# recommendations for some users
lst_user = ['A3CJ7MHAS9IMAM','A3LGT6UZL99IW1', 'A21KNRUAA5RK5E']
for i in lst_user:
  print('Recommendations for user', i, ':')
  recommendation_specific_user(i).show()

Recommendations for user A3CJ7MHAS9IMAM :
+--------+--------------+-----------------+---------+--------------+----------+
|asin_idx|reviewerID_idx|              col|   rating|    reviewerID|      asin|
+--------+--------------+-----------------+---------+--------------+----------+
|    1269|           489|[1269, 5.3275876]|5.3275876|A3CJ7MHAS9IMAM|B002TU3KTC|
|    1231|           489| [1231, 5.323815]| 5.323815|A3CJ7MHAS9IMAM|B003D7NSPG|
|    1351|           489| [1351, 5.301122]| 5.301122|A3CJ7MHAS9IMAM|B004NBZAYS|
|    1579|           489|[1579, 5.1558466]|5.1558466|A3CJ7MHAS9IMAM|B002PIUTSS|
|     872|           489|   [872, 5.15557]|  5.15557|A3CJ7MHAS9IMAM|B00358RIRC|
|    1859|           489| [1859, 5.120056]| 5.120056|A3CJ7MHAS9IMAM|B000FOO1OW|
|    2170|           489| [2170, 5.077172]| 5.077172|A3CJ7MHAS9IMAM|B00006IEA0|
|     973|           489| [973, 5.0573406]|5.0573406|A3CJ7MHAS9IMAM|B000FPGP4U|
|    1018|           489|[1018, 5.0222054]|5.0222054|A3CJ7MHAS9IMAM|B00006IFFJ

In [73]:
end_time = time.time()

In [74]:
duration = (end_time - start_time)/60
duration

7.341488639513652